In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import cv2
import torch.utils.data as data
from glob import glob
from random import shuffle
from PIL import Image
import random
from tqdm import tqdm
from pred import predprob
from skimage import io 

In [2]:
import sys
paths = sys.path
sys.path.append('/home/yuyue/yuyue/Synchronized-BatchNorm-PyTorch-master')

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((512, 512)),
#         RandomCrop((512, 512)),
        transforms.RandomHorizontalFlip(),   # horizontal flip
        transforms.RandomVerticalFlip(),   # vertival flip
        transforms.ColorJitter(0.2,0.2,0.2,0.04),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # mean, std
    ]),
    'val': transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}

In [6]:
df_train = pd.read_csv('/data/Pathology/SPIE/training_set/breastpathq/datasets/train_labels.csv') 
df_val = pd.read_csv('/data/Pathology/SPIE/training_set/breastpathq/datasets/val_labels.csv') 
df = df_train.append(df_val)

In [7]:
(df.y < 0.05).sum()

732

In [8]:
df.y[df.y > 0.049] = 1
df.y[df.y < 0.05] = 0

/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
def make_dataset(paths, extension, df):
    images = []
    for p in paths:
        if ('.'+extension) in p:
            slide = p.split('/')[-1].split('_')[0]
            slide = int(slide)
            rid = p.split('/')[-1].split('_')[1].split('.')[0]
            rid = int(rid)
            score = df[(df['slide']==slide) & (df['rid']==rid)]['y'].tolist()[0]
           #if score != 0:
            #print(p,float(score))
            images.append([p, int(score)])
    shuffle(images)
    return images

In [10]:
class SPIE_dataset(data.Dataset):
    def __init__(self, dirs, loader, extension, transform=None, train=True):
        self.samples = make_dataset(dirs, extension, df)
        if len(self.samples) == 0:
            raise(RuntimeError("no files in %s" % dirs))
        self.loader = loader
        self.transform = transform
        self.train=train
        
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        sample = Image.fromarray(sample)
        #target = torch.tensor(target).long()
        if self.transform:
            sample = self.transform(sample)
        if self.train:
            return sample, target
        else:
            return sample, target, path
        #print('target:',target)
    def __len__(self):
        return len(self.samples)

In [11]:
train_samples = glob("/data/Pathology/SPIE/training_set/breastpathq/datasets/train/*.tif")
val_samples = glob("/data/Pathology/SPIE/training_set/breastpathq/datasets/validation/*.tif")
#test_samples = torch.load( '/data/AlgProj/ydx/ydx/zhongshan/datapath/20190410_4_cls/test_444.pth')

In [12]:
train_dataset = SPIE_dataset(train_samples, io.imread, 'tif', transform=data_transforms['train'])
val_dataset = SPIE_dataset(val_samples, io.imread, 'tif', transform=data_transforms['val'])
#test_dataset = Rose_dataset(test_samples, Image.open, 'jpg', transform=data_transforms['test'], train=False)
image_datasets = {'train':train_dataset, 'val':val_dataset}
dataloaders = {"train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=16,
                                             shuffle=True, num_workers=16),
               "val": torch.utils.data.DataLoader(image_datasets["val"], batch_size=4,
                                             shuffle=True, num_workers=4)}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [13]:
def save_model(model, epoch):
    save_dir = "/data/yuyue/SPIE/model_weight/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    state_dict = model.module.state_dict()
    pth = os.path.join(save_dir, "2-classes_python2.pth")
    torch.save(state_dict, pth)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    model = DataParallel(model)
   # best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
#             count  = 0
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data
                
                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                
#                 #print("original labels = ",labels)
#                 batch_size = len(labels)
#                 class_num = 2
#                 #labels = labels.unsqueeze(1)
#                 #print(labels.shape)
#                 one_hot = torch.zeros(batch_size, class_num)
#                 for i in range(batch_size):
#                     if labels[i] == 0:
#                         one_hot[i][0] = 1
#                     else:
#                         one_hot[i][1] = labels[i]
                
#                 #print("one_hot = ",one_hot)
#                 one_hot =  Variable(one_hot.cuda())
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                outputs = model(inputs)

    
                _, preds = torch.max(outputs.data, 1) # pred值为output中最大值的位置（0是neg,1是pos）
                #_, true = torch.max(one_hot.data, 1)
#                 truth = torch.zeros((len(labels)))
#                 truth[labels>0] = 1
#                 truth = truth.long()
                
                
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.item()* inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
#                 print("labels = ",labels.cpu())
#                 print("truth = ",truth)
#                 print("preds = ",preds.cpu())
                
                
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_epoch = epoch
                save_model(model, epoch)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('Best val epoch: {:4f}'.format(best_epoch))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

In [14]:
from torch.nn import DataParallel
from densenet import densenet169
model_ft = densenet169(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 2)
from sync_batchnorm import convert_model
model_ft = convert_model(model_ft)

# load pretrained model
# val acc 0.973 @ dense169_epoch9.pth
model_ft.load_state_dict(torch.load("/data/yuyue/SPIE/model_weight/2-classes_python2.pth"))
# print("pretrained model loaded")

if use_gpu:
    model_ft = model_ft.cuda()
#weight=torch.Tensor([[1,1]]).cuda()
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# do not forget to change learning rate
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4,weight_decay=1e-4)
# Decay LR by a factor of 0.1 every 5 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=10, eta_min=0, last_epoch=-1)

densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [15]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

Epoch 0/29
----------


RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 22.38 GiB total capacity; 400.77 MiB already allocated; 80.94 MiB free; 5.48 MiB cached)